In [1]:
import subprocess
import time
import re

#### Function for command compatibility with windows cmd and some linux commands

In [2]:
def run_cmd(cmd, **kwargs):
    response = subprocess.Popen(
                    cmd, 
                    stderr=subprocess.PIPE, 
                    stdout=subprocess.PIPE, 
                    **kwargs)

    stdout, stderr = response.communicate()
    stdout = stdout.decode()
    stderr = stderr.decode()

    returncode = response.returncode

    # print('STDOUT BEGIN \n\n', stdout, '\nSTDOUT END\n\n\n')

    # If had parameter shell=True ignore bug of stderr on windows
    if kwargs.get('shell', False):
        return stdout
        
    else:
        if returncode != 0:
            print(f'Error code:   {returncode}')
            print(stderr)
        else:
            return stdout

## Verifying that devices are available

In [3]:
print(run_cmd('adb devices -l'))

cmd = 'adb devices -l | find "device " /c'
devices_connected = int(run_cmd(cmd, shell=True))
devices_connected

List of devices attached
192.168.1.67:5555      device product:beryllium model:POCOPHONE_F1 device:beryllium transport_id:9




1

In [4]:
def _devices_connected():
    cmd = 'adb devices -l | find "device " /c'
    if int(run_cmd(cmd, shell=True)) > 0:
        return True
    else:
        return False

## Obtaining the device data from the device list of the adb command

In [5]:
def get_devices_data():
    if not _devices_connected():
        return None

    cmd = 'adb devices -l'
    devices_list = run_cmd(cmd).strip().split('\n')[1:]

    for index, device in enumerate(devices_list):

        device = device.strip().split(' ')
        device.reverse()

        if device[4] != 'device':
            continue
        
        device_data = { data.split(':')[0] : data.split(':')[1] for data in device[0:4] }

        device_data['available'] = True if device[4] == 'device' else False
        device_data['serial'] = device[-1]

        devices_list[index] = device_data
    
    return devices_list

In [6]:
devices_list = get_devices_data()
devices_list

[{'transport_id': '9',
  'device': 'beryllium',
  'model': 'POCOPHONE_F1',
  'product': 'beryllium',
  'available': True,
  'serial': '192.168.1.67:5555'}]

### Connecting to the device console to get its IP address

In [7]:
serial = devices_list[0]['serial']
serial

'192.168.1.67:5555'

In [8]:
cmd = f'adb -s {serial} shell ip address'
print(run_cmd(cmd))

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host 
       valid_lft forever preferred_lft forever
2: bond0: <BROADCAST,MULTICAST,MASTER> mtu 1500 qdisc noop state DOWN group default qlen 1000
    link/ether fe:85:dc:6c:b6:3e brd ff:ff:ff:ff:ff:ff
3: dummy0: <BROADCAST,NOARP,UP,LOWER_UP> mtu 1500 qdisc noqueue state UNKNOWN group default qlen 1000
    link/ether 3e:61:78:24:f8:4a brd ff:ff:ff:ff:ff:ff
    inet6 fe80::3c61:78ff:fe24:f84a/64 scope link 
       valid_lft forever preferred_lft forever
4: ip_vti0@NONE: <NOARP> mtu 1480 qdisc noop state DOWN group default qlen 1
    link/ipip 0.0.0.0 brd 0.0.0.0
5: ip6_vti0@NONE: <NOARP> mtu 1500 qdisc noop state DOWN group default qlen 1
    link/tunnel6 :: brd ::
6: sit0@NONE: <NOARP> mtu 1480 qdisc noop state DOWN group default qlen 1


In [9]:
cmd = f'adb -s {serial} shell "ip address | grep -e \'inet .* wlan\'" '
print(run_cmd(cmd))

inet 192.168.1.67/24 brd 192.168.1.255 scope global wlan0



In [10]:
cmd = f'adb -s {serial} shell ' + '"ip address | grep -e \'inet .* wlan\' | awk \'{print $2}\'" '
cmd

'adb -s 192.168.1.67:5555 shell "ip address | grep -e \'inet .* wlan\' | awk \'{print $2}\'" '

In [11]:
ip = run_cmd(cmd)
print(ip)

ip = ip.split('/')[0]
print(ip)

192.168.1.67/24

192.168.1.67


In [12]:
def get_device_ip(serial = None):
    if serial is None:
        cmd = 'adb shell "ip address | grep -e \'inet .* wlan\' | awk \'{print $2}\'" '

    else:
        cmd = f'adb -s {serial} shell ' + '"ip address | grep -e \'inet .* wlan\' | awk \'{print $2}\'" '

    return run_cmd(cmd).split('/')[0]

In [13]:
print(get_device_ip())
print(get_device_ip(serial=serial))

192.168.1.67
192.168.1.67


In [14]:
devices_list[0]['ip'] = get_device_ip()
devices_list

[{'transport_id': '9',
  'device': 'beryllium',
  'model': 'POCOPHONE_F1',
  'product': 'beryllium',
  'available': True,
  'serial': '192.168.1.67:5555',
  'ip': '192.168.1.67'}]

In [15]:
def tcpip_connect(port = 5555, serial = None, ip = None):
    port = str(port)
    ip = get_device_ip(serial) if ip is None else ip
    
    cmd = f'adb tcpip {port}' if serial is None else f'adb -s {serial} tcpip {port}'
    print(run_cmd(cmd))

    cmd = f'adb connect {ip}:{port}'
    print(run_cmd(cmd))


In [16]:
tcpip_connect()
time.sleep(2)



already connected to 192.168.1.67:5555



In [17]:
devices_list = get_devices_data()
devices_list

In [18]:
for index, device in enumerate(devices_list):
    devices_list[index]['ip'] = get_device_ip(device['serial'])

devices_list

TypeError: 'NoneType' object is not iterable

In [19]:
sh = 'adb shell '
cmd = sh + 'find --help'

print(run_cmd(cmd))

Error code:   1
adb.exe: device still authorizing

None


In [20]:
# Searching for all the files that do not have the extension .mp3, .m4a and .flac (without distinguishing capitals)
cmd = sh + 'find sdcard/musica/ -type f ! -iname *.mp3 -a ! -iname *.m4a -a ! -iname *.flac | wc -l'

print(run_cmd(cmd))

0



In [21]:
# Deleting that files
cmd = sh + 'find sdcard/musica/ -type f ! -iname *.mp3 -a ! -iname *.m4a -a ! -iname *.flac -delete'

print(run_cmd(cmd))

In [22]:
cmd = sh + 'find sdcard/musica/ -type f ! -iname *.mp3 -a ! -iname *.m4a -a ! -iname *.flac | wc -l'

print(run_cmd(cmd))

0



In [23]:
cmd = sh + 'find sdcard/musica/ -type d -empty | wc -l'
print(run_cmd(cmd))

cmd = sh + 'find sdcard/musica/ -type d -empty'
print(run_cmd(cmd))

0




In [24]:
cmd = sh + 'find sdcard/musica/ -type d -empty -delete'
print(run_cmd(cmd))

cmd = sh + 'find sdcard/musica/ -type d -empty | wc -l'
print(run_cmd(cmd))


0



In [25]:
cmd = sh + 'ls -p sdcard/musica/AC\ DC/ -R'
artist_content = run_cmd(cmd).strip()
print(artist_content)

sdcard/musica/AC DC/:
74 Jailbreak (1984)/
AC_DC - Rock or Bust (2014) [MP3 @ 320 kbps]/
Back In Black (Australian Editions) (1980)/
Backtracks (2009)/
Ballbreaker (1995)/
Black Ice (2008)/
Blow Up Your Video (Australian Editions) (1988)/
Dirty Deeds Done Dirt Cheap (Australian Edition) (1976)/
Flick Of The Switch (Australian Editions) (1983)/
Fly On The Wall (Australian Editions) (1985)/
For Those About To Rock (We Salute You) (1981)/
High Voltage (Australian Editions) (1975)/
Highway To Hell (Australian Editions) (1979)/
If You Want Blood You've Got It (Australian Editions) (1978)/
Powerage (Australian Editions) (1978)/
Stiff Upper Lip (2000)/
T.N.T (Australian Editions) (1975)/
The Razor's Edge (Australian Editions) (1990)/
Who Made Who (Australian Editions) (1986)/
test/

sdcard/musica/AC DC/74 Jailbreak (1984):
01 - Jailbreak.mp3
02 - You Aint Got A Hold On Me.mp3
03 - Show Business.mp3
04 - Soul Stripper.mp3
05 - Baby Please Dont Go.mp3

sdcard/musica/AC DC/AC_DC - Rock or Bust (

In [26]:
artist_content = artist_content.replace('\n', '')
artist_content = artist_content.split('\r')

# deleting empty items
artist_content = [item for item in artist_content if item]

# deleting folders 
artist_content = [item for item in artist_content if item[-1] != '/' and item[-1] != ':']

artist_content.sort()
artist_content

['01 - Hard As A Rock.mp3',
 '01 - Jailbreak.mp3',
 "01 Baby, Please Don't Go.mp3",
 '01 Fly On The Wall.mp3',
 '01 For Those About To Rock (We Salute You).mp3',
 '01 Hells Bells.mp3',
 '01 Highway To Hell.mp3',
 "01 It's A Long Way To The Top (If You Wanna Rock'n'roll).mp3",
 '01 Thunderstruck.mp3',
 "01. Rock N' Roll Train.mp3",
 '01. Rock or Bust.mp3',
 '01. Stiff Upper Lip.mp3',
 "02 - Ain't No Fun (Waiting 'round To Be A Millionaire).mp3",
 '02 - Cover You In Oil.mp3',
 '02 - You Aint Got A Hold On Me.mp3',
 '02 Fire Your Guns.mp3',
 '02 Girls Got Rhythm.mp3',
 '02 Shoot To Thrill.mp3',
 '02 You Shook Me All Night Long.mp3',
 '02. Skies On Fire.mp3',
 '03 - Show Business.mp3',
 '03 - The Furor.mp3',
 "03 - There's Gonna Be Some Rockin'.mp3",
 '03 Moneytalks.mp3',
 '03 Walk All Over You.mp3',
 '04 - Boogie Man.mp3',
 '04 - Problem Child.mp3',
 '04 - Soul Stripper.mp3',
 '04 Danger.mp3',
 '04 Giving The Dog A Bone.mp3',
 '04 Go Zone.mp3',
 '04 Riff Raff.mp3',
 '04 Sink The Pink.mp3'

In [28]:
txt = artist_content[0]
re.sub(r'^\d+\s*[.-]?\s*', '', txt)

'Hard As A Rock.mp3'

In [30]:
sorted(list(map( lambda x: re.sub(r'^\d+\s*[.-]?\s*', '', x), artist_content)))

['A01 Stick Around.mp3',
 'A01 Stick Around.mp3',
 'A03 Fling Thing.mp3',
 'A03 Fling Thing.mp3',
 'A04 R.I.P. (Rock In Peace).mp3',
 'A04 R.I.P. (Rock In Peace).mp3',
 'A06 Crabsody In Blue.mp3',
 'A06 Crabsody In Blue.mp3',
 'A09 Borrowed Time.mp3',
 'A09 Borrowed Time.mp3',
 'A10 Down On The Borderline.mp3',
 'A10 Down On The Borderline.mp3',
 'A11 Big Gun.mp3',
 'A11 Big Gun.mp3',
 'A12 Cyberspace.mp3',
 'A12 Cyberspace.mp3',
 "Ain't No Fun (Waiting 'round To Be A Millionaire).mp3",
 'All Screwed Up.mp3',
 'Anything Goes.mp3',
 'Are You Ready.mp3',
 'Baby Please Dont Go.mp3',
 "Baby, Please Don't Go.mp3",
 'Back In Black.mp3',
 'Back In Business.mp3',
 'Baptism By Fire.mp3',
 'Beating Around The Bush.mp3',
 'Bedlam In Belgium.mp3',
 'Boogie Man.mp3',
 'Brain Shake.mp3',
 'Breaking The Rules.mp3',
 'C.O.D..mp3',
 'Can I Sit Next To You Girl.mp3',
 "Can't Stop Rock 'N' Roll.mp3",
 'Caught With Your Pants Down.mp3',
 'Chase The Ace.mp3',
 'Cover You In Oil.mp3',
 'Danger.mp3',
 'Decib